# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

C. Gieser  ->  C. Gieser  |  ['C. Gieser']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
F. Xu  ->  F. Xu  |  ['F. Xu']
F. Xu  ->  F. Xu  |  ['F. Xu']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
Arxiv has 86 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2601.17820


extracting tarball to tmp_2601.17820... done.


/tmp/ipykernel_3154/2822249172.py:52: LatexWarning: 2601.17820 did not run properly
Unable to locate Ghostscript on paths
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2601.18183
extracting tarball to tmp_2601.18183...

...

F. Xu  ->  F. Xu  |  ['F. Xu']


Retrieving document from  https://arxiv.org/e-print/2601.18628


extracting tarball to tmp_2601.18628...

 done.


/tmp/ipykernel_3154/2822249172.py:52: LatexWarning: 2601.18628 did not run properly
list index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.18485-b31b1b.svg)](https://arxiv.org/abs/2601.18485) | **Galactic Large-scale Filaments Resident in Asymmetric Environments: Clues from Cross-filament Profiles of Density and Temperature**  |
|| K. Su, K. Wang, <mark>F. Xu</mark>, N. K. Bhadari |
|*Appeared on*| *2026-01-27*|
|*Comments*| *27 pages, 12 figures, accepted for publication in ApJ*|
|**Abstract**|            Large-scale filaments ubiquitously exist in the Galactic interstellar medium, and their radial profiles offer insights into their formation mechanisms. We present a statistical analysis of molecular hydrogen column density ($\rm N(H_2)$) and dust temperature ($\rm T_d$) radial profiles for 35 Galactic large-scale filaments. We divided their spines into 315 segments, extracted the radial profiles of each segment using $\rm N(H_2)$ and $\rm T_d$ maps derived from $Herschel$ Hi-GAL data, and estimated the asymmetry degree within the radial profiles ($\alpha_{\rm asy}$), as well as the length proportion of segments with asymmetric profiles across the entire filament ($f_{\rm asy}$). We found that Galactic large-scale filaments reside in surroundings distinctly asymmetric and varied in $\rm N(H_2)$, and mild asymmetric yet stable in $\rm T_d$. Different filament morphology types do not show significant differences in $\alpha_{\rm asy}$ or $f_{\rm asy}$. A bent filament shape does not necessarily correspond to an asymmetric radial profile, whereas a straight filament shape may be associated with a symmetric profile. Segments with asymmetric surroundings in $\rm N(H_2)$ may not simultaneously appear asymmetric in $\rm T_d$, and vice versa. We found three filaments with 4-44% of their spine show asymmetric $\rm N(H_2)$ and $\rm T_d$ radial profiles in inverse trends, likely caused by nearby HII region. HII regions of similar scale to large filaments can induce asymmetric radial profiles within them, indicating their influence on filament evolution. However, they are unlikely to independently trigger the formation of an entire Galactic large-scale filament, in contrast to their role in small-scale filament formation.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.18183-b31b1b.svg)](https://arxiv.org/abs/2601.18183) | **Inferring photospheric horizontal flows from multiple observations with SUVEL models**  |
|| Q. Xie, J. Liu, R. Erdélyi, <mark>Y. Wang</mark> |
|*Appeared on*| *2026-01-27*|
|*Comments*| *11 pages, 4 figures, accepted by the Astrophysical Journal*|
|**Abstract**|            Photospheric horizontal velocity fields play essential roles in the formation and evolution of numerous solar activities. Various methods for estimating the horizontal velocity field have been proposed in the past. Aiming at the highest available (and future) spatial resolution (10 km/pixel) observations, a new method the Shallow U-net models (SUVEL) based on realistic numerical simulation and machine learning techniques was recently developed to track the photospheric horizontal velocity fields. Although SUVEL has been tested on numerical simulation data, its performance on solar observational data remained unclear. In this work, we apply SUVEL to the photospheric intensity observations from four ground-based solar telescopes (DKIST, GST, NVST, and SST) with the largest available apertures, and compare the results obtained from SUVEL with the Fourier local correlation tracking method (FLCT). Average correlation indices between granular regions and velocity fields inferred by SUVEL (FLCT) are 0.63, 0.81, 0.80, and 0.87 (0.00, 0.11, 0.16, and 0.10) for DKIST, GST, NVST, and SST observations. Higher correlation indices between the velocity fields tracked by SUVEL and granular patterns than FLCT reveal the superior performance of SUVEL, validating its reliability with respect to solar observational data.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.17820-b31b1b.svg)](https://arxiv.org/abs/2601.17820) | **JOYS: JWST MIRI/MRS spectra of the inner 500 au region of the L1527 IRS bipolar outflow**  |
|| R. Devaraj, et al. -- incl., <mark>C. Gieser</mark>, <mark>H. Beuther</mark>, <mark>G. Perotti</mark> |
|*Appeared on*| *2026-01-27*|
|*Comments*| *Astronomy and Astrophysics 2026*|
|**Abstract**|            This study characterizes the physical and kinematic properties within the innermost 500 au region of the L1527 bipolar outflow, a Class 0/I low-mass protostar using JWST MIRI/MRS spectroscopy across 5-28 micron at 0.2-1.0 arcsec resolution. We identify emission lines from molecular and ionized species and analyze their spatial morphology using line integrated intensity maps. We derive gas temperature and column density through excitation diagram analysis of H2 rotational lines and compared results with shock models. The observations reveal extended molecular hydrogen emission tracing the bipolar outflow, with the H2 gas temperatures distributed into warm (~550 K) and hot (~2500 K) components, likely originating from moderate velocity J-type shocks and some UV irradiation. We detect forbidden atomic and ionized emission lines of [Ni ii], [Ar ii], [Ne ii], [Ne iii], [S i], and [Fe ii] showing spatially extended morphology. Double peaked emission profiles were seen in [Ar ii], [Ne iii], and [Fe ii], in the eastern region, suggesting that the high velocity component traces a fast, highly ionized jet. Radial velocity map derived from [Ne ii] emission shows the eastern region to be redshifted and the western region blueshifted, contrary to earlier interpretations. The analysis of the MIRI/MRS observations reveals the presence of molecular, atomic, and ionized emission lines in this low-mass protostar connected with active outflow signatures. The most striking feature discovered is the presence of a poorly collimated high velocity ionized jet, embedded within a broader wide-angle molecular outflow likely driven by a disk wind. The co-existence of these components supports a stratified outflow structure and suggest L1527 exhibits unique jet-launching characteristics atypical for its early evolutionary stage.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to locate Ghostscript on paths</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.18628-b31b1b.svg)](https://arxiv.org/abs/2601.18628) | **The ALMA-ATOMS survey: Methanol emission in a large sample of hot molecular cores**  |
|| J. Zou, et al. -- incl., <mark>F. Xu</mark>, <mark>P. Garcia</mark> |
|*Appeared on*| *2026-01-27*|
|*Comments*| *Accepted for publication in The Astrophysical Journal on January 24, 2026*|
|**Abstract**|            Methanol (CH$_{3}$OH) is a key complex organic molecule (COM) in the interstellar medium, widely used as a tracer of dense gas and hot molecular cores (HMCs). Using high-resolution ALMA observations from the ATOMS survey, we investigate the excitation and abundance of methanol nuclear spin isomers and their relationship to chemical complexity in massive star-forming cores. We identify 20 methanol transitions, including A- and E-type lines in the v=0 state and E-type lines in the v$_{t}$=1 state, and detect 94 HMC candidates. Rotational temperature analysis under the LTE assumption yields average values of 194 $\pm$ 33 K for CH$_{3}$OH-E v$_{t}$=1, 178 $\pm$ 33 K for CH$_{3}$OH-A v=0, and 75 $\pm$ 33K for CH$_{3}$OH-E v=0. Emission from COMs other than methanol is detected in 87 of the 94 cores, with the CH$_{3}$OH-E v$_{t}$=1 line intensity showing a strong correlation with the channel detection ratio (CDR). These results demonstrate that CH$_{3}$OH-E v$_{t}$=1 lines are reliable tracers of HMCs and chemical complexity, and that the CDR provides a robust indicator of molecular richness. The temperature difference between A- and E-type methanol transitions is driven by anomalously strong J(2,J-2)$-$J(-1,J-1) lines, highlighting the importance of analyzing methanol symmetry types separately.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2601.18485/./fig4.png', 'tmp_2601.18485/./fig8.png', 'tmp_2601.18485/./fig5.png']
copying  tmp_2601.18485/./fig4.png to _build/html/
copying  tmp_2601.18485/./fig8.png to _build/html/
copying  tmp_2601.18485/./fig5.png to _build/html/
exported in  _build/html/2601.18485.md
    + _build/html/tmp_2601.18485/./fig4.png
    + _build/html/tmp_2601.18485/./fig8.png
    + _build/html/tmp_2601.18485/./fig5.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Galactic Large-scale Filaments Resident in Asymmetric Environments: \ Clues from Cross-filament Profiles of Density and Temperature

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.18485-b31b1b.svg)](https://arxiv.org/abs/2601.18485)<mark>Appeared on: 2026-01-27</mark> -  _27 pages, 12 figures, accepted for publication in ApJ_

</div>
<div id="authors">

K. Su, K. Wang, <mark>F. Xu</mark>, N. K. Bhadari

</div>
<div id="abstract">

**Abstract:** Large-scale filaments ubiquitously exist in the Galactic interstellar medium, and their radial profiles offer insights into their formation mechanisms. We present a statistical analysis of molecular hydrogen column density ( $\rm N(H_2)$ ) and dust temperature ( $\rm T_d$ ) radial profiles for 35 Galactic large-scale filaments. We divided their spines into 315 segments, extracted the radial profiles of each segment using $\rm N(H_2)$ and $\rm T_d$ maps derived from $Herschel$ Hi-GAL data, and estimated the asymmetry degree within the radial profiles ( $\alpha_{\rm asy}$ ), as well as the length proportion of segments with asymmetric profiles across the entire filament ( $f_{\rm asy}$ ).We found that Galactic large-scale filaments reside in surroundings distinctly asymmetric and varied in $\rm N(H_2)$ , and mild asymmetric yet stable in $\rm T_d$ .Different filament morphology types do not show significant differences in $\alpha_{\rm asy}$ or $f_{\rm asy}$ . A bent filament shape does not necessarily correspond to an asymmetric radial profile, whereas a straight filament shape may be associated with a symmetric profile.Segments with asymmetric surroundings in $\rm N(H_2)$ may not simultaneously appear asymmetric in $\rm T_d$ , and vice versa.We found three filaments with 4-44 \% of their spine show asymmetric $\rm N(H_2)$ and $\rm T_d$ radial profiles in inverse trends, likely caused by nearby H ii region. H ii regions of similar scale to large filaments can induce asymmetric radial profiles within them, indicating their influence on filament evolution. However, they are unlikely to independently trigger the formation of an entire Galactic large-scale filament, in contrast to their role in small-scale filament formation.

</div>

<div id="div_fig1">

<img src="tmp_2601.18485/./fig4.png" alt="Fig6" width="100%"/>

**Figure 6. -** (a) Distribution of the weighted average asymmetric degree $|1-\alpha_{\rm asy}|$ in the $\rm N(H_2)$ and $\rm T_d$ radial profiles of the 35 filaments, weighted by the segment length. The upper panel corresponds to the $\rm N(H_2)$ case, the lower to the $\rm T_d$ case. The x-axis shows the filaments ordered by increasing $|1-\alpha_{\rm asy, N}|$. The shape of the data points indicates the parental catalog of the filament, and color represents the morphology of the filament: L (red), C (orange), S (cyan), H (purple), and X (blue), as defined in Section \ref{subsec:filsample}. Additional gray dashed horizontal line at 0 represents the symmetric scenarios. (b) Violin plots of $|1-\alpha_{\rm asy}|$ for the five LSF morphology classes. The upper panel shows the $\rm N(H_2)$ case, the lower panel shows the $\rm T_d$ case. The violins have their corresponding morphology types labeled in the x-axis, and their colors are in the same definition as (a). The white point in each violin represents the median, and the thick black line spans the 25th and 75th percentiles. Additional gray dashed horizontal line at 0 marks symmetric scenarios. (*fig4_alpha*)

</div>
<div id="div_fig2">

<img src="tmp_2601.18485/./fig8.png" alt="Fig10" width="100%"/>

**Figure 10. -** The three LSFs with IAA segments identified as influenced by nearby Hii regions. Backgrounds from the left to right column refer to the MeerKAT spectral index map [Goedhart, et. al (2024)](https://ui.adsabs.harvard.edu/abs/2024MNRAS.531..649G), Hi-GAL $\rm N(H_2)$ and $\rm T_d$ map  ([Marsh, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.471.2730M/abstract)) . The red crosses and black segments connecting them represent the filament spine, segments colored in magenta have IAA radial profiles. Dashed boxes mark the radial profile sampling regions of each segment. Circles represent nearby WISE Hii regions, orange ones matches the filament distance within its uncertainty range, while cyan circles either differ in distance or lack distance information. A 5 pc scale bar is included in the lower right corner of each panel. (*fig8_IAAfil*)

</div>
<div id="div_fig3">

<img src="tmp_2601.18485/./fig5.png" alt="Fig7" width="100%"/>

**Figure 7. -** (a) Estimated $f_{\rm asy}$ based on the $\rm N(H_2)$ and $\rm T_d$ radial profiles of all 35 LSFs. The upper panel corresponds to the $\rm N(H_2)$ case, and the lower panel to the $\rm T_d$ case. The x-axis arranges the filaments by increasing $f_{\rm asy, N}$. Data point shapes and colors follow the same definitions as in Figure \ref{fig4_alpha}. Dashed gray dashed horizontal lines at 0 and 1 represent fully symmetric and fully asymmetric surroundings, respectively. (b) Violin plots of $f_{\rm asy}$ for the five LSF morphology classes, with the $\rm N(H_2)$ case in the upper panel, and the $\rm T_d$ case in the lower panel. Morphology labels and color conventions are the same as in Figure \ref{fig4_alpha}. Dashed gray horizontal lines at 0 and 1 represent fully symmetric and asymmetric surroundings, respectively. (*fig5_fasy*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.18485"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

114  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
